In [ ]:
#------------------------------
# Face Recognition with OpenCV
#------------------------------

import cv2
import os
import numpy as np

#------------------------------
# Training Data
#------------------------------
# As OpenCV face recognizer accepts labels as integers so we need to define a mapping between integer labels and persons actual names so below I am defining a mapping of persons integer labels and their respective names. 
#there is no label 0 in our training data so subject name for index/label 0 is empty
subjects = ["None","0001","0002","0003","0004","0005","0006","0007","0008","0009","0010"]

# Prepare training data
# 
# 1. Read all the folder names of subjects/persons provided in training data folder
# 2. For each subject, extract label number. Folder names follow the format `sLabel` where `Label` is an integer representing the label we have assigned to that subject.
# 3. Read all the images of the subject, detect face from each image.
# 4. Add each face to faces vector with corresponding subject label (extracted in above step) added to labels vector. 
#------------------------------
#Face detection function
#------------------------------
def detect_face(img):
    #convert the test image to gray image as opencv face detector expects gray images
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #load OpenCV LBP face detector
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5);
    
    #if no faces are detected then return original img
    if (len(faces) == 0):
        return None, None
    
    #Extract face area
    (x, y, w, h) = faces[0]
    
    #return only the face part of the image
    return gray[y:y+w, x:x+h], faces[0]


#------------------------------
# Training Data Preparation
#------------------------------

#this function will read all persons' training images, detect face from each image
#and will return one list of faces and another list of labels for each face
def prepare_training_data(data_folder_path):
    
    #get the directories (one directory for each subject) in data folder
    dirs = os.listdir(data_folder_path)
    
    #list to hold all subject faces
    faces = []
    #list to hold labels for all subjects
    labels = []
    
    #let's go through each directory and read images within it
    for dir_name in dirs:
        
        if not dir_name.startswith("s"):
            continue;
            
        #extract label number of subject from dir_name
        label = int(dir_name.replace("s", ""))
        
        #build path of directory containin images for current subject subject
        subject_dir_path = data_folder_path + "/" + dir_name
        
        #get the images names that are inside the given subject directory
        subject_images_names = os.listdir(subject_dir_path)
        
        #go through each image name, read image, 
        #detect face and add face to list of faces
        for image_name in subject_images_names:
            
            #ignore hidden files
            if image_name.startswith("."):
                continue;
            
            #build image path
            image_path = subject_dir_path + "/" + image_name

            #read image
            image = cv2.imread(image_path)
            
            #display an image window to show the image 
            cv2.imshow("Training on image...", cv2.resize(image, (400, 500)))
            cv2.waitKey(100)
            
            #detect face
            face, rect = detect_face(image)

            if face is not None:
           #add face to list of faces
              faces.append(face)
           #add label for this face
              labels.append(label)
            
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    
    return faces, labels

#prepare our training data
#data will be in two lists of same size
#one list will contain all the faces
#and other list will contain respective labels for each face
print("Preparing data...")
faces, labels = prepare_training_data("train-data")
print("Data prepared")

#print total faces and labels
print("Total faces: ", len(faces))
print("Total labels: ", len(labels))

#------------------------------
# Train Face Recognizer
#------------------------------
# Used LBPH face recognizer

#create LBPH face recognizer 
face_recognizer = cv2.face.LBPHFaceRecognizer_create()

#train our face recognizer of our training faces
face_recognizer.train(faces, np.array(labels))

#----------------------------------
# Draw rectangle on identified face
#----------------------------------
#function to draw rectangle on image 
#according to given (x, y) coordinates and 
#given width and heigh
def draw_rectangle(img, rect):
    (x, y, w, h) = rect
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)

#-----------------------------------
# Draw label text on identified face
#-----------------------------------
#function to draw text on give image starting from (x, y) coordinates. 
def draw_text(img, text, x, y):
    cv2.putText(img, text, (x-5, y), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2)


#------------------------------
# Face prediction
#------------------------------
#this function recognizes the person in image passed and draws a rectangle around detected face with name of the subject
def predict(test_img):
    img = test_img.copy()
    #detect face from the image
    face, rect = detect_face(img)

    #predict the image using our face recognizer 
    label, confidence = face_recognizer.predict(face)
    print("label",label," Confidence",confidence)
    
    #get name of respective label returned by face recognizer
    label_text = subjects[label]
    
    #draw a rectangle around face detected
    draw_rectangle(img, rect)
    #draw name of predicted person
    draw_text(img, label_text, rect[0], rect[1]-5)
    
    return img

#------------------------------
# Test Images using prediction
#------------------------------

print("Predicting images...")

#load test images
test_img1 = cv2.imread("test-data/test1.jpg")
test_img2 = cv2.imread("test-data/test2.jpg")
#test_img3 = cv2.imread("test-data/test3.JPG")
test_img4 = cv2.imread("test-data/test4.jpg")

#perform a prediction
predicted_img1 = predict(test_img1)
predicted_img2 = predict(test_img2)
#predicted_img3 = predict(test_img3)
predicted_img4 = predict(test_img4)
print("Prediction complete")

#display both images
cv2.imshow(subjects[1], cv2.resize(predicted_img1, (400, 500)))
cv2.imshow(subjects[2], cv2.resize(predicted_img2, (400, 500)))
#cv2.imshow(subjects[3], cv2.resize(predicted_img3, (400, 500)))
cv2.imshow(subjects[6], cv2.resize(predicted_img4, (400, 500)))
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)
cv2.destroyAllWindows()

Preparing data...
Data prepared
Total faces:  38
Total labels:  38
Predicting images...
label 1  Confidence 0.0
label 2  Confidence 0.0
label 6  Confidence 6.715962045346678
Prediction complete
